# Export hostnames of Telnet data

The issue with Telnet data is, that the Telnet service sometimes logs the hostname
and sometimes the IP address of the requesting host. Unfortunately, the hostname
is not always properly assignable.

In [ ]:
APP_NAME = "Extract hostnames of Telnet"

In [ ]:
%load configuration.py

In [ ]:
%load setup.py

In [ ]:
%load data.py

In [ ]:
df_telnet = {}

for i in range(len(dates) - 1):
    print(f"Load {dates[i]} to {dates[i+1]}")
    df_telnet[dates[i]] = df_log.select("timestamp", "raw").filter("category == 'telnet'").filter(f"timestamp >= '{dates[i]}' and timestamp < '{dates[i+1]}'")

#df_telnet[dates[0]].show(20);


In [ ]:
from pyspark.sql import functions as fn
import ipaddress

path = "/home/max/jupyter/dataplane/export/"
#df.select("raw").split("' from '")

def is_host(s):
    try:
        ipaddress.ip_address(s)
        return False
    except:
        return True

def df_to_list(df):
    df = df.select(fn.split("raw", "' from '")[1].alias("raw"))
    df = df.select(fn.split("raw", "' FOR '")[0].alias('host'))
    df = df.distinct()
    arr = df.collect()
    arr = [r.host for r in arr]
    arr = [x for x in arr if is_host(x)]
    return arr

#tmp = df.select(fn.split("raw", "' from '")[1].alias('raw'))
#tmp = tmp.select(fn.split("raw", "' FOR '")[0].alias('host'))
#tmp = tmp.distinct()
#tmp = tmp.select(fn.map_filter("host", is_ip).alias("host"))
#tmp = tmp.select(
#    fn.filter("host", is_ip).alias("host")
#)
#tmp = tmp.filter(tmp.host)

#tmp = tmp.collect()
#tmp = [r.host for r in tmp]
#tmp = list(filter(is_host, tmp))

#print(tmp)

hosts = {}

for x in dates:
    print(f"Dataframe to host list {x}")
    if x in df_telnet.keys():
        hosts[x] = df_to_list(df_telnet[x])


for k, v in hosts.items():
    _f = f"{path}/hosts-{k.year}-{k.month}-{k.day}.txt"
    
    print(f"Write {len(v)} hosts to file {_f}")
    with open(_f, 'w') as f:
        f.writelines([x + '\n' for x in v])